In [32]:
import requests


1. (Python/R): Для белка P04637 (p53) получите список его функций (features)


In [58]:
url = "https://www.ebi.ac.uk/proteins/api/features"
params = {'accession': 'P04637'} 

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()  
    print(f"Protein: {data[0]['entryName']}")
    print(f"Accession: {data[0]['accession']}")
    print(f"Sequence: {data[0]['sequence']}")
else:
    print("Error:", response.status_code)


Protein: P53_HUMAN
Accession: P04637
Sequence: MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD


2. (Python/R): Получите последовательности белков P05067 (APP) и P10636 (Tau). Сравните их длины. Какой белок длиннее и на сколько аминокислот?


In [34]:
def get_uniprot_seq_length(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    response = requests.get(url)
    seq = ''.join([line.strip() for line in response.text.splitlines() if not line.startswith('>')])
    return len(seq)

len_app = get_uniprot_seq_length("P05067")
len_tau = get_uniprot_seq_length("P10636")

if len_app > len_tau:
    longer = "APP (P05067)"
    diff = len_app - len_tau
elif len_tau > len_app:
    longer = "Tau (P10636)"
    diff = len_tau - len_app
else:
    longer = "Одинаковые по длине"
    diff = 0

print(f"APP (P05067): {len_app} a.a.")
print(f"Tau (P10636): {len_tau} a.a.")
print(f"Длиннее белок: {longer}, на {diff} аминокислот")


APP (P05067): 770 a.a.
Tau (P10636): 758 a.a.
Длиннее белок: APP (P05067), на 12 аминокислот


3. (Python/R): Получите данные о 5 различных белках (на ваш выбор) через API, используя их accession IDs. Сохраните их entryName и длину последовательности в таблицу (data frame в R/Pandas)


In [35]:
import pandas as pd

accessions = ['P04637', 'P05067', 'P10636', 'P02649', 'P02452']  
results = []

for acc in accessions:
    url = f'https://rest.uniprot.org/uniprotkb/{acc}.json'
    resp = requests.get(url)
    data = resp.json()
    entry = data['uniProtkbId'] if 'uniProtkbId' in data else 'N/A'
    length = data['sequence']['length']
    results.append({'accession': acc, 'entryName': entry, 'length': length})

df = pd.DataFrame(results)
df.to_csv('proteins.csv', index=False)
print(df)

  accession    entryName  length
0    P04637    P53_HUMAN     393
1    P05067     A4_HUMAN     770
2    P10636    TAU_HUMAN     758
3    P02649   APOE_HUMAN     317
4    P02452  CO1A1_HUMAN    1464


4.(Python/R): Используя API PubMed, найдите 10 самых последних (сортировка по дате) статей по запросу "COVID-19 variants". Выведите их ID и заголовки.

In [36]:
import requests

base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    'db': 'pubmed',
    'term': 'COVID-19 variants',
    'retmode': 'json',
    'retmax': 10 
}

data = requests.get(base_url, params=params).json()
id_list = data['esearchresult']['idlist']

# получить заголовки по ID
summary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
summary = requests.get(summary_url, params={'db':'pubmed', 'id':','.join(id_list), 'retmode':'json'}).json()

print("\nID и заголовки:")
for uid in summary['result']['uids']:
    print(uid, "-", summary['result'][uid]['title'])


ID и заголовки:
41184328 - Targeting asparagine and cysteine in SARS-CoV-2 variants and human pro-inflammatory mediators to alleviate COVID-19 severity; a cross-section and in-silico study.
41184286 - SARS-CoV-2 NSP14 inhibitor exhibits potent antiviral activity and reverses NSP14-driven host modulation.
41183191 - Modulating antigen processing through metal-organic frameworks to bias adaptive immunity.
41183040 - Integrating genomic epidemiology and deep mutational scanning data for prevalence forecasting of SARS-CoV-2 Omicron lineages.
41181333 - Evaluating lymphocyte change rate and lactate as predictors of prognosis in critical COVID-19 patients in the intensive care unit.
41181326 - Comparative plasma metabolomics of Delta and Omicron SARS-CoV-2 variants: insights into variant-specific pathogenesis and therapeutic implications.
41181315 - COVID-19 infection and return-to-play outcomes of elite athletes in Japan: a retrospective descriptive study.
41181283 - Cost-Effectiveness Ana

5. (Python): Напишите функцию get_protein_name(accession), которая принимает на вход accession ID белка и возвращает его название (entryName). Обработайте случай, если белок не найден.

In [37]:


def get_protein_name(accession):
    url = f'https://rest.uniprot.org/uniprotkb/{accession}.json'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            entry_name = data.get("uniProtkbId", None)
            return entry_name if entry_name is not None else "Название не найдено"
        else:
            return "Белок не найден"
    except Exception as e:
        return f"Ошибка запроса: {e}"

In [38]:
print(get_protein_name("P04637"))

P53_HUMAN


In [39]:
print(get_protein_name("P0463"))

Белок не найден


6.(Python): Прочитайте из файла accessions.txt (нужно создать) список из 3-5 accession IDs. Для каждого ID получите из Proteins API и запишите в новый файл его название и организм-источник.

In [40]:
import pandas as pd

with open('accessions.txt', 'r') as f:
    accessions = [line.strip() for line in f if line.strip()]


data = []
for acc in accessions:
    url = f'https://rest.uniprot.org/uniprotkb/{acc}.json'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            result = response.json()
            entry = result.get('uniProtkbId', 'N/A')
            organism = result.get('organism', {}).get('scientificName', 'N/A')
    except:
        entry = 'error'
        organism = 'error'
    data.append({'accession': acc, 'entryName': entry, 'organism': organism})

df = pd.DataFrame(data)
df.to_csv('proteins_info.csv', index=False)

print(df)


  accession    entryName      organism
0    P04637    P53_HUMAN  Homo sapiens
1    P05067     A4_HUMAN  Homo sapiens
2    P10636    TAU_HUMAN  Homo sapiens
3    P02649   APOE_HUMAN  Homo sapiens
4    P02452  CO1A1_HUMAN  Homo sapiens


7.(Python/R): Для найденных в задании 4 статей получите более подробную информацию, используя Eutils API и ID статей (конечная точка efetch.fcgi). Извлеките имена авторов для каждой статьи.


In [41]:
ids = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={"db":"pubmed","term":"COVID-19 variants","retmode":"json","retmax":10}
).json()["esearchresult"]["idlist"]

for pmid in ids:
    txt = requests.get(
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
        params={"db":"pubmed","id":pmid,"rettype":"medline","retmode":"text"}
    ).text
    authors = [line[6:].strip() for line in txt.splitlines() if line.startswith("AU  - ")]
    print(pmid, "→", "; ".join(authors) if authors else "(нет авторов)")


41184328 → Fadel HH; El-Esseily HA; Ahmed MAE; Khamis Mohamed MA; Roushdy MN; ElSherif A; Gharbeya KM; Abdelsalam HS
41184286 → Luo M; Mo J; Wang Z; Wei H; Chen K; Shen L; Wang Y; Li L; Chen Y; Chen W; Li X; Feng H; Wang X; Zhou H; Li B; Xu F; Zhao Q; Xu Y; Che J; Zou P; Zhang R; Dong X; Xie W
41183191 → Cho E; Davis MA; Nowak JA; Izzo M; Ferrante AM; Sha F; Magdalenski JS; Farha OK; Teplensky MH
41183040 → Lei ZY; Zhang XM; Han JL; Xue JG; Xu JY; Ren ZL; Tong YG; Bo XC; Ni M
41181333 → Tang Y; Yang J; Chen L; Liu X; Chen Z; Lin J; Jin J; Wei Y
41181326 → Pimentel E; Banoei MM; Lee CH; Winston BW
41181315 → Fukushima K; Kamahara K; Tomori A; Nakata Y
41181283 → Valverde HA; Yacapin CC; Pasco PMD; Real JOM; Lopez JCF; Clapham HE; Chew YZ; Mercado CEG; Botwright S; Silzle M; Lam HY
41177465 → Morwitzer MJ; Zheng YY; Friberg H; Currier JR
41177007 → Lee J; Sung D; Woo CH; Kim BG; Lee JW


8. (Bash): Напишите команду curl, которая получает данные о белке, выполняет поиск по слову "phosphorylation" (либо другое на ваш выбор) в ответе и выводит строки, содержащие это слово. 
Сделала для белка P04637 (p53) из первого задания


curl -s "https://rest.uniprot.org/uniprotkb/P04637.txt" | grep -i phosphorylation

9. Белок Q8WZ42 (Titin):  получить последовательность и разбить на строки по 80 символов


curl -sS "https://rest.uniprot.org/uniprotkb/Q8WZ42.fasta" | awk 'NR==1{next} {seq=seq $0} END {for(i=1;i<=length(seq);i+=80) print substr(seq,i,80)}'

11. (Python/R): Напишите скрипт, который для заданного гена (например, APOE) находит белки у человека, получает их ID, названия и ссылки на соответствующие записи в UniProt. Сохраните результат в CSV-файл.


In [60]:
import pandas as pd

gene = "APOE"  
r = requests.get(
    "https://rest.uniprot.org/uniprotkb/search",
    params={
        "query": f"gene_exact:{gene} AND organism_id:9606",
        "fields": "accession,id,protein_name,organism_name",
        "format": "json",
        "size": 500
    },
)
r.raise_for_status()
res = r.json().get("results", [])

rows = []
for x in res:
    acc   = x["primaryAccession"]
    entry = x["uniProtkbId"]
    name  = (x.get("proteinDescription", {}).get("recommendedName", {}) or {}) \
            .get("fullName", {}).get("value", "")
    org   = x.get("organism", {}).get("scientificName", "")
    rows.append({
        "Gene": gene,
        "Accession": acc,
        "EntryName": entry,
        "ProteinName": name,
        "Organism": org,
        "UniProtURL": f"https://www.uniprot.org/uniprotkb/{acc}",
    })

df = pd.DataFrame(rows)
df.to_csv(f"{gene}_proteins.csv", index=False)
print(df)


    Gene   Accession         EntryName       ProteinName      Organism  \
0   APOE      P02649        APOE_HUMAN  Apolipoprotein E  Homo sapiens   
1   APOE  A0A0S2Z3D5  A0A0S2Z3D5_HUMAN  Apolipoprotein E  Homo sapiens   
2   APOE      E7ERP7      E7ERP7_HUMAN  Apolipoprotein E  Homo sapiens   
3   APOE      H0Y7L5      H0Y7L5_HUMAN  Apolipoprotein E  Homo sapiens   
4   APOE      J9ZVQ3      J9ZVQ3_HUMAN  Apolipoprotein E  Homo sapiens   
5   APOE      D9ZB55      D9ZB55_HUMAN  Apolipoprotein E  Homo sapiens   
6   APOE  A0A346DBY2  A0A346DBY2_HUMAN  Apolipoprotein E  Homo sapiens   
7   APOE      E9PEV4      E9PEV4_HUMAN  Apolipoprotein E  Homo sapiens   
8   APOE  A0A0S2Z3B1  A0A0S2Z3B1_HUMAN                    Homo sapiens   
9   APOE  A0A0S2Z3J5  A0A0S2Z3J5_HUMAN                    Homo sapiens   
10  APOE      Q8TCZ8      Q8TCZ8_HUMAN                    Homo sapiens   
11  APOE  A0A0S2Z3V0  A0A0S2Z3V0_HUMAN                    Homo sapiens   

                                     

12. (Python): Ваш коллега написал код, который в цикле делает 100 запросов к API (по 1 белку). Перепишите код, чтобы сделать всего 1 запрос, получающий данные сразу по 100 белкам (используйте соответствующий параметр в Proteins API)


In [43]:
url = "https://rest.uniprot.org/uniprotkb/search"
params = {
    "query": "reviewed:true AND organism_id:9606",  # человек
    "format": "json",
    "size": 100  
}

r = requests.get(url, params=params)
data = r.json()

for p in data["results"]:
    print(p["primaryAccession"], "-", p["proteinDescription"]["recommendedName"]["fullName"]["value"])



A0A0C5B5G6 - Mitochondrial-derived peptide MOTS-c
A0A1B0GTW7 - Ciliated left-right organizer metallopeptidase
A0JNW5 - Bridge-like lipid transfer protein family member 3B
A0JP26 - POTE ankyrin domain family member B3
A0PK11 - Clarin-2
A1A4S6 - Rho GTPase-activating protein 10
A1A519 - Protein FAM170A
A1L190 - Synaptonemal complex central element protein 3
A1L3X0 - Very long chain fatty acid elongase 7
A1X283 - SH3 and PX domain-containing protein 2B
A2A2Y4 - FERM domain-containing protein 3
A2RU14 - Transmembrane protein 218
A2RUB6 - Coiled-coil domain-containing protein 66
A2RUC4 - tRNA wybutosine-synthesizing protein 5
A4D1B5 - Gamma-secretase-activating protein
A4GXA9 - Structure-specific endonuclease subunit EME2
A5D8V7 - Outer dynein arm-docking complex subunit 3
A5PLL7 - Plasmanylethanolamine desaturase 1
A6BM72 - Multiple epidermal growth factor-like domains protein 11
A6H8Y1 - Transcription factor TFIIIB component B'' homolog
A6NCS4 - Homeobox protein Nkx-2.6
A6NFY7 - Succinate

14. Создайте сценарий, который целенаправленно вызывает ошибку 404 при работе с API, и корректно их обрабатывает с выводом информативных сообщений.


In [64]:
url = "https://api.example.com/nonexistent_endpoint"

try:
    response = requests.get(url)
    if response.status_code == 404:
        print("Ошибка 404.")
    else:
        response.raise_for_status()
        print("Ответ получен:", response.json())
except requests.exceptions.RequestException as e:
    print("Ошибка  запроса:", e)

Ошибка  запроса: HTTPSConnectionPool(host='api.example.com', port=443): Max retries exceeded with url: /nonexistent_endpoint (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7a834962d630>: Failed to resolve 'api.example.com' ([Errno -2] Name or service not known)"))


15.  Напишите скрипт, который принимает название гена, ищет по нему белки у человека, а затем для первого найденного белка ищет статьи в PubMed, упоминающие его ген. Выведите ID статей.


In [31]:
import requests

gene = input("Gene: ").strip()

# 1) UniProt: первый белок человека по гену
u = "https://rest.uniprot.org/uniprotkb/search"
params = {"query": f"gene_exact:{gene} AND organism_id:9606", "format": "json", "size": 1}
r = requests.get(u, params=params, timeout=15)
r.raise_for_status()
j = r.json()

if not j.get("results"):
    print("Белки не найдены для Homo sapiens.")
else:
    res = j["results"][0]
    gene_name = (res.get("genes", [{}])[0].get("geneName", {}) or {}).get("value", gene)

    # 2) PubMed: ищем статьи, где упоминается этот ген (у человека)
    p = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    pq = {"db": "pubmed", "retmode": "json",
          "term": f"\"{gene_name}\"[Title/Abstract] AND Homo sapiens[Organism]"}
    pm = requests.get(p, params=pq, timeout=15)
    pm.raise_for_status()
    ids = pm.json().get("esearchresult", {}).get("idlist", [])

    print("Gene:", gene_name)
    print("PMIDs:", " ".join(ids) if ids else "(ничего не найдено)")


Gene:  APOE


Gene: APOE
PMIDs: 41163688 41163083 41157904 41155526 41154661 41153431 41153344 41153028 41152198 41152152 41148185 41148136 41147554 41146472 41134938 41134879 41121436 41121378 41114585 41114448


17. Выберите любой интересующий вас белок или ген. Самостоятельно придумайте и выполните 3 различных запроса к Proteins API или PubMed, которые дадут о нем новую информацию. Опишите, что вы узнали.


In [66]:

url = "https://rest.uniprot.org/uniprotkb/search"
params = {
    "query": "gene_exact:COL2A1 AND organism_id:9606 AND reviewed:true",
    "format": "json",
    "fields": "accession,id,protein_name,length,organism_name",
    "size": 1,
}
r = requests.get(url, params=params); r.raise_for_status()
data = r.json()
rec = data["results"][0]

print("Accession:", rec["primaryAccession"])
print("Entry:", rec["uniProtkbId"])
print("Protein name:", rec["proteinDescription"]["recommendedName"]["fullName"]["value"])
print("Length (aa):", rec["sequence"]["length"])
print("Organism:", rec["organism"]["scientificName"])

Accession: P02458
Entry: CO2A1_HUMAN
Protein name: Collagen alpha-1(II) chain
Length (aa): 1487
Organism: Homo sapiens


In [69]:
url = "https://rest.uniprot.org/uniprotkb/search"
params = {
    "query": "gene_exact:COL2A1 AND organism_id:9606 AND reviewed:true",
    "format": "json",
    "fields": "cc_function,cc_subcellular_location",
    "size": 1,
}
rec = requests.get(url, params=params).json()["results"][0]

function = " ".join(
    t.get("value", "") 
    for c in rec.get("comments", []) if c.get("commentType") == "FUNCTION"
    for t in c.get("texts", [])
).strip() or "—"

locs = {
    (loc.get("location") or {}).get("value") 
    for c in rec.get("comments", []) 
    for loc in c.get("locations", [])
    if (loc.get("location") or {}).get("value")
}

print("Function:", function)


Function: Type II collagen is specific for cartilaginous tissues. It is essential for the normal embryonic development of the skeleton, for linear growth and for the ability of cartilage to resist compressive forces


In [68]:
esearch = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        "db": "pubmed",
        "term": "(COL2A1[Title/Abstract]) AND (cartilage[Title/Abstract]) AND (2023:3000[dp])",
        "retmode": "json",
        "retmax": 5,
        "sort": "most recent",
    },
)
esearch.raise_for_status()
ids = esearch.json().get("esearchresult", {}).get("idlist", [])

if not ids:
    print("Статей не найдено.")
else:
    esum = requests.get(
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",
        params={"db": "pubmed", "id": ",".join(ids), "retmode": "json"},
    )
    esum.raise_for_status()
    result = esum.json().get("result", {})
    for pmid in ids:
        title = result.get(pmid, {}).get("title", "(без заголовка)")
        print(f"{pmid}: {title}")

41172600: Dual-loaded BMSCs-Metformin microgels promote cartilage repair through combined anti-inflammatory, antioxidant, and regenerative effects.
41163191: Optimizing LED photobiomodulation parameters to prevent cartilage matrix degradation in knee osteoarthritis: in vitro and in vivo study.
41155522: Characterization of Recombinant Human Type II Collagen from CHO Cells, Functional Assessment of Chondrocytes and Alleviation of Cartilage Degeneration.
41155076: Chondrogenic Maturation Governs hMSC Mechanoresponsiveness to Dynamic Compression.
41126931: Biomechanical and biological features of hyaluronic acid in combination with chondroitin and platelet rich plasma for regenerative medicine applications.


Коллаген альфа-1(II) цепи (Accession: P02458, Entry: CO2A1_HUMAN) — это фибриллярный белок, ключевой компонент для хрящевой ткани человека. Он играет важную роль в нормальном развитии скелета и росте, а также обеспечивает прочность и упругость хряща, помогая сопротивляться сдавливающим нагрузкам. Коллаген типа II преимущественно встречается в хрящах, стекловидном теле глаза, внутреннем ухе и межпозвонковых дисках позвоночника. Ряд научных исследований подтверждают его важность для регенерации и защиты хрящевой ткани при различных состояниях, включая остеоартрит и повреждения суставов.


16.  (Bash): Напишите bash-скрипт, который с помощью curl и jq (если установлен) извлекает accession ID из заранее сохраненного JSON-файла с данными белка и выводит его.


In [54]:
jq -r '.accession' protein.json

SyntaxError: unterminated string literal (detected at line 6) (4249831771.py, line 6)